# cpp
参考:  
https://en.cppreference.com/w/cpp/thread/shared_mutex  
https://murphypei.github.io/blog/2019/04/cpp-concurrent-1  
https://zhuanlan.zhihu.com/p/194198073  
https://glumes.com/post/c++/c++-multi-thread/ 待研究 


In [ ]:
#include <iostream>
#include <thread>
#include <mutex> // 互斥锁
#include <shared_mutex> // 读写锁
#include <chrono>
using namespace std;


mutex m;            // 实例化互斥锁m对象，不要理解为定义变量
shared_mutex s_m;   // 实例化读写锁对象
int num = 1;        // 共享资源

// 函数
void plus1(int num) // 输入+2，不修改输入的值
{
    num += 2;
    std::this_thread::sleep_for(std::chrono::milliseconds(5)); // 给定时长，阻塞当前线程
    cout << "输入num in 子线程id: " << this_thread::get_id() << " = " << num << endl;
}

void plus2(int& num) // & 与 std::ref()相对应，输入+2，修改输入的值
{
    num += 2;
    std::this_thread::sleep_for(std::chrono::milliseconds(10)); // 给定时长，阻塞当前线程
    cout << "输入num in 子线程id: " << this_thread::get_id() << " = " << num << endl;
}

void plus3(int delta) // 共享资源+2，修改共享资源的值
{
    num += delta;
    std::this_thread::sleep_for(std::chrono::milliseconds(15)); // 给定时长，阻塞当前线程
    cout << "共享资源 num in 子线程plus3 id: " << this_thread::get_id() << " = " << num << endl;
}

void multiply(int a) // 修改共享资源的值
{
    num *= a;
    std::this_thread::sleep_for(std::chrono::milliseconds(7)); // 给定时长，阻塞当前线程
    cout << "共享资源 num in 子线程multiply id: " << this_thread::get_id() << " = " << num << endl;
}

// 可调用对象
class foo
{
public:
    void count() // 修改foo.n
    {
        for (int i = 0; i < 4; ++i) {
            ++n;
            std::this_thread::sleep_for(std::chrono::milliseconds(3));
        }
    }
    int n = 0;
};


// 互斥锁
void plus_mutex(int delta)
{
    m.lock();
    num += delta;
    //std::this_thread::sleep_for(std::chrono::milliseconds(1)); // 给定时长，阻塞当前线程
    cout << "共享资源 num in 子线程plus_mutex id: " << this_thread::get_id() << " = " << num << endl;
    m.unlock();
}

void multiply_mutex(int a)
{
    m.lock();
    num *= a;
    //std::this_thread::sleep_for(std::chrono::milliseconds(2)); // 给定时长，阻塞当前线程
    cout << "共享资源 num in 子线程multiply_mutex id: " << this_thread::get_id() << " = " << num << endl;
    m.unlock();
}

// lock_guard
void plus_lock_guard(int delta)
{
    {
        lock_guard<mutex> g1(m);//用此语句替换了m.lock()；lock_guard传入一个参数时，该参数为互斥量，此时调用了lock_guard的构造函数，申请锁定m
        num += delta;
        cout << "共享资源 num in 子线程plus_lock_guard id: " << this_thread::get_id() << " = " << num << endl;

    }//通过使用{}来调整作用域范围，可使得m在合适的地方被解锁
}

void multiply_lock_guard(int a)
{
    {
        lock_guard<mutex> g2(m);
        num *= a;
        cout << "共享资源 num in 子线程multiply_lock_guard id: " << this_thread::get_id() << " = " << num << endl;
    }
}

// 读写锁
void plus_shared_mutex(int delta)
{
    s_m.lock();
    num += delta;
    std::this_thread::sleep_for(std::chrono::milliseconds(5)); // 给定时长，阻塞当前线程
    cout << "共享资源 num in 子线程plus_shared_mutex id: " << this_thread::get_id() << " = " << num << endl;
    s_m.unlock();
}

void read()
{
    s_m.lock_shared();
    std::this_thread::sleep_for(std::chrono::milliseconds(1)); // 给定时长，阻塞当前线程
    cout << "共享资源 num in 子线程read id: " << this_thread::get_id() << " = " << num << endl;
    s_m.unlock_shared();
}

int main()
{
    int y = 0;
    foo f1;
    int input1 = 1;
    int input2 = 1;

    // 比较传入value和reference 对输入及共享资源的变化
    std::thread th1(plus1, input1);           // 函数, pass by value，不改变输入的值
    std::thread th2(plus2, std::ref(input2)); // 函数, pass by reference，改变输入的值
    std::thread th3(plus3, 2);              // 函数, pass by value，改变共享资源的值
    std::thread th4(&foo::count, &f1);      // 可调用对象
    cout << "主线程中显示子线程th1的id: " << th1.get_id() << endl;

    th1.join(); // 等待线程th1完成，同步
    th2.join();
    th3.join();
    th4.join();

    cout << "input1 in main = " << input1 << endl;
    cout << "input2 in main = " << input2 << endl;
    cout << "num in main = " << num << endl;
    std::cout << "f1.n (foo::n) = " << f1.n << std::endl;


    // 无锁,进程会同时读写共享资源
    cout << "\ncase: 无锁" << endl;
    num = 1;
    for (int ii = 0; ii < 10; ii++)
    {
        thread thx(plus3, 1);
        thread thy(multiply, 2);
        thx.join();
        thy.join();
    }

    // 互斥锁，进程有序读写共享资源
    cout << "\ncase: 互斥锁/mutex" << endl;
    // 互斥就是保证资源同一时刻只能被一个进程使用；互斥是为了保证数据的一致性，
    // 如果A线程在执行计算式A的时候，某个量被B线程改掉了，这可能会出现问题，
    // 于是要求资源互斥，我在用它你就不能用，等我用完了你再用，我们彼此互不干扰。
    num = 1;
    cout << endl;
    for (int ii = 0; ii < 10; ii++)
    {
        thread thx(plus_mutex, 1);
        thread thy(multiply_mutex, 2);
        thx.join();
        thy.join();
    }

    cout << "\ncase: lock_guard<mutex>" << endl;
    // 其原理是：声明一个局部的std::lock_guard对象，在其构造函数中进行加锁，在其析构函数中进行解锁。
    // 最终的结果就是：创建即加锁，作用域结束自动解锁。从而使用std::lock_guard()就可以替代lock()与unlock()。
    // 通过设定作用域，使得std::lock_guard在合适的地方被析构（在互斥量锁定到互斥量解锁之间的代码叫做临界区（需要互斥访问共享资源的那段代码称为临界区），
    // 临界区范围应该尽可能的小，即lock互斥量后应该尽早unlock），通过使用{}来调整作用域范围，可使得互斥量m在合适的地方被解锁
    num = 1;
    cout << endl;
    for (int ii = 0; ii < 10; ii++)
    {
        thread thx(plus_lock_guard, 1);
        thread thy(multiply_lock_guard, 2);
        thx.join();
        thy.join();
    }

    // 读写锁，进程同时读共享资源，有序写共享资源
    cout << "\ncase: 读写锁/shared_mutex" << endl;
    // shared_mutex读写锁把对共享资源的访问者划分成读者和写者，多个读线程能同时读取共享资源，但只有一个写线程能同时读取共享资源
    num = 1;
    for (int ii = 0; ii < 10; ii++)
    {
        thread thx(plus_shared_mutex, 1);
        thread thy(read);
        thx.join();
        thy.join();
    }

    //getchar(); // 让程序停留在这一步,直到它从键盘接收到消息

    return 0;
}

输出

In [ ]:
主线程中显示子线程th1的id: 3912
输入num in 子线程id: 3912 = 3
输入num in 子线程id: 3248 = 3
共享资源 num in 子线程plus3 id: 30756 = 3
input1 in main = 1
input2 in main = 3
num in main = 3
f1.n (foo::n) = 4

case: 无锁
共享资源 num in 子线程multiply id: 26772 = 3
共享资源 num in 子线程plus3 id: 14688 = 3
共享资源 num in 子线程multiply id: 28004 = 8
共享资源 num in 子线程plus3 id: 24984 = 8
共享资源 num in 子线程multiply id: 30956 = 18
共享资源 num in 子线程plus3 id: 29556 = 18
共享资源 num in 子线程multiply id: 14900 = 38
共享资源 num in 子线程plus3 id: 22116 = 38
共享资源 num in 子线程multiply id: 5564 = 77
共享资源 num in 子线程plus3 id: 9636 = 77
共享资源 num in 子线程multiply id: 23436 = 155
共享资源 num in 子线程plus3 id: 9088 = 155
共享资源 num in 子线程multiply id: 17432 = 311
共享资源 num in 子线程plus3 id: 14640 = 311
共享资源 num in 子线程multiply id: 26348 = 624
共享资源 num in 子线程plus3 id: 26136 = 624
共享资源 num in 子线程multiply id: 30632 = 1250
共享资源 num in 子线程plus3 id: 30076 = 1250
共享资源 num in 子线程multiply id: 17164 = 2501
共享资源 num in 子线程plus3 id: 24788 = 2501

case: 互斥锁/mutex

共享资源 num in 子线程multiply_mutex id: 9684 = 2
共享资源 num in 子线程plus_mutex id: 23320 = 3
共享资源 num in 子线程multiply_mutex id: 31476 = 6
共享资源 num in 子线程plus_mutex id: 31716 = 7
共享资源 num in 子线程plus_mutex id: 28780 = 8
共享资源 num in 子线程multiply_mutex id: 19808 = 16
共享资源 num in 子线程plus_mutex id: 27736 = 17
共享资源 num in 子线程multiply_mutex id: 24932 = 34
共享资源 num in 子线程plus_mutex id: 26924 = 35
共享资源 num in 子线程multiply_mutex id: 18636 = 70
共享资源 num in 子线程multiply_mutex id: 26472 = 140
共享资源 num in 子线程plus_mutex id: 17568 = 141
共享资源 num in 子线程multiply_mutex id: 22768 = 282
共享资源 num in 子线程plus_mutex id: 26804 = 283
共享资源 num in 子线程multiply_mutex id: 32680 = 566
共享资源 num in 子线程plus_mutex id: 9508 = 567
共享资源 num in 子线程multiply_mutex id: 18364 = 1134
共享资源 num in 子线程plus_mutex id: 15460 = 1135
共享资源 num in 子线程plus_mutex id: 4780 = 1136
共享资源 num in 子线程multiply_mutex id: 6380 = 2272

case: lock_guard<mutex>

共享资源 num in 子线程multiply_lock_guard id: 27424 = 2
共享资源 num in 子线程plus_lock_guard id: 8272 = 3
共享资源 num in 子线程multiply_lock_guard id: 6600 = 6
共享资源 num in 子线程plus_lock_guard id: 29608 = 7
共享资源 num in 子线程multiply_lock_guard id: 29716 = 14
共享资源 num in 子线程plus_lock_guard id: 18996 = 15
共享资源 num in 子线程multiply_lock_guard id: 29240 = 30
共享资源 num in 子线程plus_lock_guard id: 1048 = 31
共享资源 num in 子线程multiply_lock_guard id: 7144 = 62
共享资源 num in 子线程plus_lock_guard id: 15032 = 63
共享资源 num in 子线程plus_lock_guard id: 27540 = 64
共享资源 num in 子线程multiply_lock_guard id: 13732 = 128
共享资源 num in 子线程multiply_lock_guard id: 23288 = 256
共享资源 num in 子线程plus_lock_guard id: 26676 = 257
共享资源 num in 子线程multiply_lock_guard id: 9900 = 514
共享资源 num in 子线程plus_lock_guard id: 32528 = 515
共享资源 num in 子线程multiply_lock_guard id: 15204 = 1030
共享资源 num in 子线程plus_lock_guard id: 12812 = 1031
共享资源 num in 子线程plus_lock_guard id: 29932 = 1032
共享资源 num in 子线程multiply_lock_guard id: 27980 = 2064

case: 读写锁/shared_mutex
共享资源 num in 子线程read id: 15100 = 1
共享资源 num in 子线程plus_shared_mutex id: 19380 = 2
共享资源 num in 子线程read id: 14560 = 2
共享资源 num in 子线程plus_shared_mutex id: 26288 = 3
共享资源 num in 子线程read id: 22596 = 3
共享资源 num in 子线程plus_shared_mutex id: 17940 = 4
共享资源 num in 子线程plus_shared_mutex id: 1176 = 5
共享资源 num in 子线程read id: 14352 = 5
共享资源 num in 子线程read id: 27672 = 5
共享资源 num in 子线程plus_shared_mutex id: 13096 = 6
共享资源 num in 子线程read id: 30228 = 6
共享资源 num in 子线程plus_shared_mutex id: 30936 = 7
共享资源 num in 子线程plus_shared_mutex id: 2620 = 8
共享资源 num in 子线程read id: 22912 = 8
共享资源 num in 子线程plus_shared_mutex id: 21112 = 9
共享资源 num in 子线程read id: 31784 = 9
共享资源 num in 子线程plus_shared_mutex id: 30580 = 10
共享资源 num in 子线程read id: 16204 = 10
共享资源 num in 子线程read id: 31464 = 10
共享资源 num in 子线程plus_shared_mutex id: 31468 = 11